In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from google.colab import files
uploaded = files.upload()

Saving training_data.csv to training_data (1).csv


In [0]:
import io
train_data = pd.read_csv(io.BytesIO(uploaded['training_data.csv']))
# Dataset is now stored in a Pandas Dataframe

In [0]:
#train_data = pd.read_csv('training_data.csv')
train_data = pd.DataFrame(train_data)

In [0]:
for i in range(len(train_data)):
    if (train_data.loc[i,"10board"]=="0"):
        train_data.loc[i,"10board"]="others"
    if (train_data.loc[i,"12board"]=="0"):
        train_data.loc[i,"12board"]="others"

In [0]:
train_data[["10percentage"]]=train_data[["10percentage"]].fillna(train_data[["10percentage"]].mean())

In [0]:
train_data[["conscientiousness"]]=train_data[["conscientiousness"]].fillna(train_data[["conscientiousness"]].mean())


In [0]:
train_data[["nueroticism"]]=train_data[["nueroticism"]].fillna(train_data[["nueroticism"]].mean())

In [0]:
for i in range(len(train_data)):
    if (train_data.loc[i,"JobCity"]=="-1"):
        train_data.loc[i,"JobCity"]="others"
    train_data.loc[i,"JobCity"]=train_data.loc[i,"JobCity"].lower()
#training_data["JobCity"].nunique()
    
        

In [0]:
train_data["12graduation"].fillna(3000, inplace = True) 
for i in range(3000):
    if(train_data.loc[i,"12graduation"] == 3000):
        if(train_data.loc[i,"Degree"] == "B.Tech/B.E."):
            train_data.loc[i,"12graduation"] = train_data.loc[i,"GraduationYear"] - 4
        if(train_data.loc[i,"Degree"] == "M.Tech/M.E."):
            train_data.loc[i,"12graduation"] = train_data.loc[i,"GraduationYear"] - 2
        if(train_data.loc[i,"Degree"] == "MCA"):
            train_data.loc[i,"12graduation"] = train_data.loc[i,"GraduationYear"] - 6

In [0]:
#train_data[["12graduation","Degree","GraduationYear"]]
train_data["Specialization"].fillna(0, inplace = True) 
train_data["JobCity"].fillna(0, inplace = True) 
train_data["CollegeCityID"].fillna(0, inplace = True)

for i in range(3000):
    if(train_data.loc[i,"Specialization"] == 0):
        train_data.loc[i,"Specialization"] = "other"
    if(train_data.loc[i,"JobCity"] == 0):
        train_data.loc[i,"JobCity"] = "other" 
    if(train_data.loc[i,"CollegeCityID"] == 0):
        train_data.loc[i,"CollegeCityID"] = 20000   
    if(train_data.loc[i,"ComputerProgramming"] == -1):
        train_data.loc[i,"ComputerProgramming"] = 0  
    if(train_data.loc[i,"ElectronicsAndSemicon"] == -1):
        train_data.loc[i,"ElectronicsAndSemicon"] = 0 
    if(train_data.loc[i,"ComputerScience"] == -1):
        train_data.loc[i,"ComputerScience"] = 0 
    if(train_data.loc[i,"MechanicalEngg"] == -1):
        train_data.loc[i,"MechanicalEngg"] = 0 
    if(train_data.loc[i,"ElectricalEngg"] == -1):
        train_data.loc[i,"ElectricalEngg"] = 0 
    if(train_data.loc[i,"TelecomEngg"] == -1):
        train_data.loc[i,"TelecomEngg"] = 0 
    if(train_data.loc[i,"CivilEngg"] == -1):
        train_data.loc[i,"CivilEngg"] = 0 
    if(train_data.loc[i,"Domain"] == -1):
        train_data.loc[i,"Domain"] = 0 

In [0]:
train_data["Gender"]

0       m
1       m
2       m
3       m
4       m
       ..
2995    f
2996    m
2997    m
2998    m
2999    m
Name: Gender, Length: 3000, dtype: object

In [0]:
#train_data__onehot = train_data.copy()
train_data = pd.get_dummies(train_data, columns=['Gender','Degree'], prefix = ['Gender','Degree'])

print(train_data.head())

   Unnamed: 0  ID  ...  Degree_M.Tech./M.E. Degree_MCA
0           0   0  ...                    0          1
1           1   1  ...                    0          0
2           2   2  ...                    0          0
3           3   3  ...                    0          0
4           4   4  ...                    0          0

[5 rows x 40 columns]


In [0]:
train_data= train_data.drop(["ID","10board","12board","JobCity","DOB","CollegeState","Specialization"],axis=1)

In [0]:
train_data.shape


(3000, 33)

In [0]:
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

X_train=train_data.iloc[:,2:].values
y=train_data.iloc[:,1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_train,y,test_size=0.2, random_state=0)

y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)


from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)
y_train.values.reshape(-1,1)
y_test.values.reshape(-1,1)
y_train=sc.fit_transform(y_train)
y_test=sc.transform(y_test)

from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor(n_estimators=150,random_state=0)
regressor.fit(X_train,y_train)
y_pred=regressor.predict(X_test)
y_pred_train=regressor.predict(X_train)

from sklearn import metrics
from sklearn.metrics import r2_score
#print("Root mean squared error:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
#print("Root mean squared error:",np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))
print("R2 score:",metrics.r2_score(y_test, y_pred))





/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


R2 score: 0.0691399878366441


In [0]:
#from sklearn import datasets
import xgboost as xgb

xgdmat=xgb.DMatrix(X_train,y_train)

our_params={'eta':0.3,'seed':0,'subsample':0.8,'colsample_bytree':0.8,'objective':'reg:linear','max_depth':6,'min_child_weight':1}

final_gb=xgb.train(our_params,xgdmat)

tesdmat=xgb.DMatrix(X_test)

y_pred=final_gb.predict(tesdmat)
#print(y_pred)

from sklearn.metrics import mean_squared_error
import math
#testScore=math.sqrt(mean_squared_error(y_test,y_pred))
#print(testScore)
print("R2 score:",metrics.r2_score(y_test, y_pred))


[17:08:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Root mean squared error: 0.058996916929238585


In [0]:
from sklearn.linear_model import Ridge

ridgeReg = Ridge(alpha=0.05, normalize=True)

ridgeReg.fit(X_train,y_train)

y_pred = ridgeReg.predict(X_test)

print("R2 score:",metrics.r2_score(y_test, y_pred))

R2 score: 0.08048578131927053


In [0]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

# Create the dataset
rng = np.random.RandomState(1)


# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=6),
                          n_estimators=200, random_state=rng)

regr_1.fit(X_train, y_train)
regr_2.fit(X_train, y_train)

# Predict
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

print("R2 score:",metrics.r2_score(y_test, y_1))
print("R2 score:",metrics.r2_score(y_test, y_2))



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R2 score: 0.04116033766415084
R2 score: 0.07506291285452993


In [0]:
from sklearn.linear_model import Lasso

lassoReg = Lasso(alpha=0.01, normalize=True)

lassoReg.fit(X_train,y_train)

y_pred = lassoReg.predict(X_test)

print("R2 score:",metrics.r2_score(y_test, y_pred))





R2 score: -5.9177684518241236e-05
